# Bedrock Knowledge Base Retrieval and Generation with SageMaker Inference and Metadata Filtering  

### Description:  
This notebook showcases how to query and retrieve information from an Amazon Bedrock-powered knowledge base while leveraging SageMaker inference and metadata filtering. It covers key steps such as configuring queries, applying metadata filters, retrieving responses, and extracting citations used in the generated results.

![Metadata Filtering](./metadata_filtering.png)

## 1. Import and Load Configuration Variables

In [ ]:
# Import required functions from advanced_rag_utils
from advanced_rag_utils import (
    load_variables, 
    setup_bedrock_client,
    create_standard_filter,
    retrieve_from_bedrock_with_filter,
    format_llama3_prompt,
    generate_sagemaker_response,
    retrieve_and_generate_with_sagemaker
)

# Load configuration variables
variables = load_variables("../variables.json")
variables  # Display the loaded variables for confirmation

## 2. Set Up Required IDs and Model ARNs

In [ ]:
# Knowledge Base Selection  
kb_id = variables["kbFixedChunk"]  # Options: "kbFixedChunk", "kbHierarchicalChunk", "kbSemanticChunk"

# Retrieval-Augmented Generation (RAG) Configuration  
number_of_results = 3  # Number of relevant documents to retrieve  
generation_configuration = {
    "temperature": 0,  # Lower temperature for more deterministic responses  
    "top_k": 10,  # Consider top 10 tokens at each generation step  
    "max_new_tokens": 5000,  # Maximum number of tokens to generate  
    "stop": "<|eot_id|>"  # Stop sequence to end the response generation  
}

# User Query
query = "what was the % increase in sales?"  # Sample query to retrieve data from the knowledge base

# SageMaker endpoint
endpoint_name = variables['sagemakerLLMEndpoint']

# Initialize Bedrock client
bedrock_client = setup_bedrock_client(variables["regionName"])

## 3. Define Metadata Filter

In [ ]:
# Create a standard filter for document type and year
metadata_filter = create_standard_filter('10K Report', 2023)

## 4. Generate Response with Metadata Filter

In [ ]:
# Use the combined retrieve and generate function
response, context = retrieve_and_generate_with_sagemaker(
    query=query,
    knowledge_base_id=kb_id,
    sagemaker_endpoint=endpoint_name,
    metadata_filter=metadata_filter,
    generation_config=generation_configuration,
    bedrock_client=bedrock_client,
    num_results=number_of_results,
    region_name=variables["regionName"]
)

# Print the user's query
print("Question:", {query})

# Uncomment below line if you want to debug and see the retrieved context
# print(f"Context: {context}")

# Print the generated answer
print("Answer:", response)

## 5. (Optional) Step-by-Step Approach

In [ ]:
# If you prefer to execute the steps individually:

# 1. Retrieve context from Bedrock KB with metadata filtering
context = retrieve_from_bedrock_with_filter(
    query=query,
    knowledge_base_id=kb_id,
    metadata_filter=metadata_filter,
    bedrock_client=bedrock_client,
    num_results=number_of_results,
    region_name=variables["regionName"]
)

# 2. Format prompt using retrieved context
prompt = format_llama3_prompt(query, context)

# 3. Generate response using SageMaker endpoint
response = generate_sagemaker_response(
    prompt=prompt,
    endpoint_name=endpoint_name,
    generation_config=generation_configuration
)

# 4. Display results
print("Question:", {query})
# print(f"Context: {context}")  # Uncomment for debugging
print("Answer:", response)